In [1]:
import gensim
from gensim.models import KeyedVectors
from stellargraph.data import EdgeSplitter
import networkx as nx
from sklearn.linear_model import LogisticRegressionCV
import pandas as pd
import pickle

2023-03-01 18:31:56.970025: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-01 18:31:57.165060: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-01 18:31:57.171499: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-01 18:31:57.171534: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
model = gensim.models.Word2Vec.load('../lm-vol/2_28_new_structure_500_epochs_word2vec.model')
wv = KeyedVectors.load('../lm-vol/2_28_new_structure_500_epochs_word2vec.wordvectors', mmap='r')
G = nx.read_graphml('../lm-vol/LANL_train_unique_v2.graphml')

In [3]:
def operator_hadamard(u, v):
    return u * v
node_embeddings = (wv.vectors) 
# Define an edge splitter on the original graph:
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from graph, and obtain the
# reduced graph graph_test with the sampled links removed:
graph_test, edges_train, labels_train = edge_splitter_test.train_test_split(
    p=0.99, method="global"
)
training_edges = [operator_hadamard(wv[edge[0]], wv[edge[1]]) for edge in edges_train]

** Sampled 971898 positive and 971898 negative edges. **


In [5]:
clf = LogisticRegressionCV(cv=5, max_iter=5000, random_state=0).fit(training_edges, labels_train)
clf.score(training_edges, labels_train)

0.7116430942341686

In [6]:
pkl_filename = "../lm-vol/2_28_new_structure_500_epochs_lr.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)